In [4]:
import openai 
from langchain.text_splitter import CharacterTextSplitter


In [60]:
import pandas as pd 
df = pd.read_csv('Mahabharat_Metadata.csv')
df

,Id,Book,Section_Id,File_Name,URL,Section_Roman,Word_Count,Char_Count,Para_Count,Word_Len
0,1,Book 1: Adi Parva,Book01_01,mahabharata_01002.txt,https://www.mahabharataonline.com/translation/...,Section I,7464,44206,45,5.92
1,2,Book 1: Adi Parva,Book01_02,mahabharata_01003.txt,https://www.mahabharataonline.com/translation/...,Section II,8608,52123,33,6.06
2,3,Book 1: Adi Parva,Book01_03,mahabharata_01004.txt,https://www.mahabharataonline.com/translation/...,Section III,6089,35041,48,5.75
3,4,Book 1: Adi Parva,Book01_04,mahabharata_01005.txt,https://www.mahabharataonline.com/translation/...,Section IV,341,1991,4,5.84
4,5,Book 1: Adi Parva,Book01_05,mahabharata_01006.txt,https://www.mahabharataonline.com/translation/...,Section V,876,5068,11,5.79
...,...,...,...,...,...,...,...,...,...,...
2095,2102,Book 18: Svargarohanika Parva,Book18_02,mahabharata_18002.txt,https://www.mahabharataonline.com/translation/...,Section 2,1410,8312,13,5.90
2096,2103,Book 18: Svargarohanika Parva,Book18_03,mahabharata_18003.txt,https://www.mahabharataonline.com/translation/...,Section 3,1164,6676,7,5.74
2097,2104,Book 18: Svargarohanika Parva,Book18_04,mahabharata_18004.txt,https://www.mahabharataonline.com/translation/...,Section 4,608,3519,5,5.79
2098,2105,Book 18: Svargarohanika Parva,Book18_05,mahabharata_18005.txt,https://www.mahabharataonline.com/translation/...,Section 5,1668,10059,13,6.03


# Create Question

- This is very expensive stuff for 29 mn tokens of Mahabharat. Which will cost around $250 just fow question creation. So droping this idea.

In [57]:
# import openai
# from langchain.callbacks import get_openai_callback
# openai.api_key = secret_key

# qtn_info={}

# # Generate questions based on the context supplied. Multiple contexts are created from section text.
# def get_questions(context_id, no_of_questions, context):
#     # try:
#     prompt_ = f"""Write {no_of_questions} questions and their corresponding answers based on the text below\n\n
#     Text: {context}\n\n
#     Questions: \n\n
#     Answer:\n"""
#     print(prompt_)

#     with get_openai_callback() as cb:
#         response = openai.Completion.create(
#             engine="davinci-instruct-beta-v3",
#             prompt=prompt_,
#             temperature=0,
#             max_tokens=257,
#             top_p=1,
#             frequency_penalty=0,
#             presence_penalty=0,
#             stop=["\n\n"]
#         )

#         # Get the number of tokens used
#         prompt_tokens_ = cb.prompt_tokens
#         completion_tokens_ = cb.completion_tokens

#         # Get the total cost of the API call
#         total_cost_ = cb.total_cost
        
#         print(f"ContextId: {context_id}, PToken: {prompt_tokens_}, CToken: {completion_tokens_}, Cost: {total_cost_}")
#         qtn_info[context_id] = f"PToken: {prompt_tokens_}, CToken: {completion_tokens_}, Cost: {total_cost_}"

#         questions =  response  #"1." + response['choices'][0]['text']
#     # except:
#     #     questions= ""

#     return questions, cb

# Create Chunk and Prompt for Questions Generation

- Book 8,9,10,11, 16,17,18 Don't have chapters. They are being handled differently.

In [78]:

def Generate_QA_Prompt(df1, df_prompts, idx_of_files, context_id):
    for i in idx_of_files:
        section_file_name = df1.loc[i,"File_Name"]
        section_id = df1.loc[i, "Section_Id"]
        words_in_section = df1.loc[i, "Word_Count"] 
        chars_in_section = df1.loc[i,"Char_Count"]
        #question_to_generate = df1.loc[i,"Questions"]

        print ("File:",section_file_name,
                '| Sectionid:', section_id,
                "| Words:", words_in_section, 
                "| CharCount:",chars_in_section,
                #'| Questions:',question_to_generate
                )
            
            
        section_file_path = r"MBBook-Txt/"+ section_file_name
        with open(section_file_path, "r") as file_section:
            text = file_section.readlines()
            text = "\n".join(text)
            # text = text.replace('\n\n \n\n', '\n')

        # split into chunks
        text_splitter = CharacterTextSplitter(
            separator="\n",
            chunk_size=1800,
            chunk_overlap=200,
            length_function=len
        )
                        
        chunks = text_splitter.split_text(text)
        for chunk in chunks:
            # prompt_ = f"""Write 1 question, corresponding answer, and their reference from the context based on the text below\n\nText: {chunk}\n\nQuestions: \n\nAnswer: \n\nReference:\n\n"""

            prompt_ = f"""Write 1 question, corresponding answer, and their reference from the context based on the text below (do not repeat the context in your response)\n\nText: {chunk}\n\n"""

            new_row = section_id, context_id, prompt_ 
            df_prompts.loc[context_id] = new_row
            context_id = context_id +1

    return context_id, df_prompts


In [ ]:
# Initialize dataset for prompt 
df_prompts = pd.DataFrame(columns=['Section_Id','Context_Id','Prompt'])
# Initialize context_id
context_id = 1 

# Generate Questions & Answers for Sections.
idx_of_files = df.index
context_id, df_prompts = Generate_QA_Prompt(df, df_prompts, idx_of_files, context_id)
print(f"{context_id} Contexts Generated")


In [80]:
context_id

7363

In [75]:
df_prompts.to_csv('Question_Prompts-2000char-Context.csv',index=None)

In [72]:
df_prompts.groupby('Section_Id').count()['Prompt'].reset_index().to_csv('Number_of_QA_Prompts.csv', index=None)

# Ignore the Code Below

In [ ]:
# # Initialize dataset for prompt 
# df_prompts = pd.DataFrame(columns=['SectionId','ContextId','Prompt'])
# # Initialize context_id
# context_id = 1 

# # Generate Questions & Answers for Sections.
# df1 = df[df['SectionName'].str.contains('Mahabharata')]
# idx_of_files = df1.index
# context_id, df_prompts = Generate_QA_Prompt(df1, df_prompts, idx_of_files, context_id)
# print(f"{context_id} Contexts Generated")

# # Generate Questions & Answers for those Books which does not have Sections.
# df1 = df[df['id'].isin(['08', '09', '10','11', '16','17','18'])]
# idx_of_files = df1.index
# context_id, df_prompts = Generate_QA_Prompt(df1, df_prompts, idx_of_files, context_id)
# print(f"{context_id} Contexts Generated")

In [58]:
section_id = 1
context_id = 1
text = chunks[2]
questions, cb = get_questions(context_id, 5, text)

# #df['questions']= df.context.apply(get_questions)

('Write 5 questions and their corresponding answers based on the text below\n\n\nText: The son of Satyavati having, by penance and meditation, analysed the eternal Veda, afterwards composed this holy history, when that learned Brahmarshi of strict vows, the noble Dwaipayana Vyasa, offspring of Parasara, had finished this greatest of narrations, he began to consider how he might teach it to his disciples. And the possessor of the six attributes, Brahma, the world\'s preceptor, knowing of the anxiety of the Rishi Dwaipayana, came in person to the place where the latter was, for gratifying the saint, and benefiting the people. And when Vyasa, surrounded by all the tribes of Munis, saw him, he was surprised; and, standing with joined palms, he bowed and ordered a seat to be brought. And Vyasa having gone round him who is called Hiranyagarbha seated on that distinguished seat stood near it; and being commanded by Brahma Parameshthi, he sat down near the seat, full of affection and smiling i

ContextId: 1, PToken: 0, CToken: 0, Cost: 0.0


In [59]:
cb

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0

In [60]:
questions

<OpenAIObject text_completion id=cmpl-7bSpghAWj3ySAPyaFBJUGWOruyOFk at 0x1f8bd3f6d40> JSON: {
  "id": "cmpl-7bSpghAWj3ySAPyaFBJUGWOruyOFk",
  "object": "text_completion",
  "created": 1689163324,
  "model": "davinci-instruct-beta-v3",
  "choices": [
    {
      "text": "\n1. What did Vyasa ask Ganesa to do?\n2. What did Ganesa agree to do?\n3. What did Vyasa say to Ganesa about his writing?\n4. How many verses does Sauti know?\n5. What does the wisdom of the Bharata do?",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1202,
    "completion_tokens": 69,
    "total_tokens": 1271
  }
}

In [46]:
for q in questions.split('\n'):
    print (q)

1. Who is the author of the text?
2. What is the tree of the Bharata?
3. What is the Bharata?
4. What is the significance of the Bharata?
5. Who are the Pandavas?


In [ ]:
i=1
for chunk in chunks:
    print (i,'----\n',chunk)
    i+=1

In [ ]:
dirname = r'D:\github\1-Projects\Research\SSBM\Booksummarization\Mahabharat\MBBook-Sections'
for f in os.listdir(dirname):
    newname = f.replace("-","_")
    oldname = f 
    os.rename( os.path.join(dirname,oldname), os.path.join(dirname,newname))
    print(newname)


In [ ]:
df_questions = pd.DataFrame(columns=['Question_Id', 'Context_Id', 'Section_Id', 'Question'])

section_id = '01_001'
context_id = 1
qid=1
for question in questions.split('\n'):
    new_row = qid, context_id, section_id, question 
    df_questions.loc[qid] = new_row
    qid = qid +1

df_questions

In [ ]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    # Your OpenAI API call here
    response = openai.Completion.create(
        engine="davinci",
        prompt="Hello, world!",
        max_tokens=5,
        n=1,
        stop=None,
        temperature=0.5,
    )

    # Get the number of tokens used
    prompt_tokens = cb.prompt_tokens
    completion_tokens = cb.completion_tokens

    # Get the total cost of the API call
    total_cost = cb.total_cost

    print(f"Prompt Tokens: {prompt_tokens}")
    print(f"Completion Tokens: {completion_tokens}")
    print(f"Total Cost (USD): $ {total_cost}")


In [ ]:
def getChunks(text):
  # split into chunks
  text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
  )
  chunks = text_splitter.split_text(text)
  return chunks 

def getText(filename):
  with open(filename, "r") as file:
    text = "\n".join(file.readlines())
    return text 

In [19]:


def get_questions(context, n):
    #return ("First Question")
    try:
      prompt = PromptTemplate(
        input_variables=["context", "qtn_numbers"],
        template= "Write {qtn_numbers} unique questions based on the text below\n\nText: {context}\n\nQuestions:\n1.",

      llm = OpenAI(engine="davinci-instruct-beta-v3",
          temperature=0.1,
          max_tokens=257,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0,
          stop=["\n\n"])
      
    #   chain = LLMChain(llm=llm, chain_type="stuff", prompt=prompt)
      chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt)

      with get_openai_callback() as cb:
          response = chain.run(context=context, qtn_numbers=n)
          print(cb.tot)
          
      return response['choices'][0]['text']
    except:
        return ""

In [ ]:
def get_questions(context):
    try:
      response = openai.Completion.create(
          engine="davinci-instruct-beta-v3",
          prompt=f"Write questions based on the text below\n\nText: {context}\n\nQuestions:\n1.",
          temperature=0,
          max_tokens=257,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0,
          stop=["\n\n"]
      )
      return response['choices'][0]['text']
    except:
        return ""

Created a chunk of size 10833, which is longer than the specified 4000


File: Mahabharata_BOOK_01_ADI_PARVA_SECTION_001-I.txt Words: 7882 CharCount: 44701 Questions: 5.0


In [ ]:


        # create embeddings
        embeddings = OpenAIEmbeddings()
        knowledge_base = FAISS.from_texts(chunks, embeddings)
        
        # show user input
        user_question = st.text_input("Ask a question about your PDF:")
        if user_question:
            docs = knowledge_base.similarity_search(user_question)
            
            llm = OpenAI()
            chain = load_qa_chain(llm, chain_type="stuff")
            with get_openai_callback() as cb:
                response = chain.run(input_documents=docs, question=user_question)
                print(cb)
            
            st.write(response)

# Get Answers

In [38]:
import os 

'module\\chapter'

In [ ]:
# %%writefile app.py



def getAnswer(fileName, Question):
    load_dotenv()

  
    filename = os.path.join("MBBook-Sections",filename)
    text = getText(filename)
        
    chunks = getChunks(text)
      
    # create embeddings
    embeddings = OpenAIEmbeddings()
    knowledge_base = FAISS.from_texts(chunks, embeddings)
      
      # show user input
      user_question = 
      if user_question:
        docs = knowledge_base.similarity_search(user_question)
        
        llm = OpenAI()
        chain = load_qa_chain(llm, chain_type="stuff")
        with get_openai_callback() as cb:
          response = chain.run(input_documents=docs, question=user_question)
          print(cb)
           
        st.write(response)

def generateAnswers():
  for i in df:
    if df.loc[i, "Type"] != "Book": #Get Questions only from Sections.
        print ("File:",df.loc[i,"SectionName"],
                'id:', df.loc[i, "id"],
                "Words:", df.loc[i, "WordCount"], 
                "CharCount:",df.loc[i,"CharCount"],
                'Questions:',df.loc[i,"Questions"])
        
        NumOfQuestions = df.loc[i,"Questions"]
        fname = r"MBBook-Sections/"+ df.loc[i,"SectionName"]
        id = df.loc[i, "id"]

        # filename = 'Mahabharata_BOOK_01_ADI_PARVA_SECTION_001-I.txt
        # question = 3
        # id = '01_001'
        # getAnswer(fname,NumOfQuestions,id)

generateAnswers()


# Experiments

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(engine="davinci-instruct-beta-v3", temperature=0.1, max_tokens=257,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0)

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("colorful socks"))





Soxariety.


In [ ]:
prompt = PromptTemplate(
    input_variables=["company", "product"],
    template="What is a good name for {company} that makes {product}?",
)
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({
    'company': "ABC Startup",
    'product': "colorful socks"
    }))



Fancy Footwear.


In [ ]:
import openai

# Set up your OpenAI API credentials
openai.api_key = secret_key #'YOUR_API_KEY'

# Step 1: Generate a set of prompts or instructions for Langchain
context_document = text

langchain_prompts = [
    "Generate question based on the given context.",
    "Please provide a question that relates to the provided context.",
    #"Inquiry: Generate a question based on the provided information.",
    # Add more prompts as desired
]

# Step 2: Use Langchain to generate questions based on the context
generated_questions = []

for prompt in langchain_prompts:
    response = openai.Completion.create(
        engine='text-davinci-003',  # Choose the appropriate language model
        prompt=f"{context_document}\n\nPrompt: {prompt}",
        max_tokens=50,  # Set the desired length for generated questions
        n = 3,  # Number of questions to generate per prompt (adjust as needed)
        stop=None,  # Optional: Specify a stop sequence to end the question generation
        temperature=0.7,  # Adjust the temperature parameter as desired
        echo=True  # Set to True for interactive conversations
    )

    for choice in response.choices:
        generated_questions.append(choice.text.strip())

# Step 3: Display the generated questions
for i, question in enumerate(generated_questions, 1):
    print(f"Question {i}: {question}")


In [32]:
text = """  SECTION I

 Om! Having bowed down to Narayana and Nara, the most exalted male being, and also to the goddess Saraswati, must the word Jaya be uttered.

 Ugrasrava, the son of Lomaharshana, surnamed Sauti, well-versed in the Puranas, bending with humility, one day approached the great sages of rigid vows, sitting at their ease, who had attended the twelve years' sacrifice of Saunaka, surnamed Kulapati, in the forest of Naimisha. Those ascetics, wishing to hear his wonderful narrations, presently began to address him who had thus arrived at that recluse abode of the inhabitants of the forest of Naimisha. Having been entertained with due respect by those holy men, he saluted those Munis (sages) with joined palms, even all of them, and inquired about the progress of their asceticism. Then all the ascetics being again seated, the son of Lomaharshana humbly occupied the seat that was assigned to him. Seeing that he was comfortably seated, and recovered from fatigue, one of the Rishis beginning the conversation, asked him, 'Whence comest thou, O lotus-eyed Sauti, and where hast thou spent the time? Tell me, who ask thee, in detail.'

 Accomplished in speech, Sauti, thus questioned, gave in the midst of that big assemblage of contemplative Munis a full and proper answer in words consonant with their mode of life.

 "Sauti said, 'Having heard the diverse sacred and wonderful stories which were composed in his Mahabharata by Krishna- Dwaipayana, and which were recited in full by Vaisampayana at the Snake-sacrifice of the high-souled royal sage Janamejaya and in the presence also of that chief of Princes, the son of Parikshit, and having wandered about, visiting many sacred waters and holy shrines, I journeyed to the country venerated by the Dwijas (twice-born) and called Samantapanchaka where formerly was fought the battle between the children of Kuru and Pandu, and all the chiefs of the land ranged on either side. Thence, anxious to see you, I am come into your presence. Ye reverend sages, all of whom are to me as Brahma; ye greatly blessed who shine in this place of sacrifice with the splendour of the solar fire: ye who have concluded the silent meditations and have fed the holy fire; and yet who are sitting--without care, what, O ye Dwijas (twice-born), shall I repeat, shall I recount the sacred stories collected in the Puranas containing precepts of religious duty and of worldly profit, or the acts of illustrious saints and sovereigns of mankind?"

 "The Rishi replied, 'The Purana, first promulgated by the great Rishi Dwaipayana, and which after having been heard both by the gods and the Brahmarshis was highly esteemed, being the most eminent narrative that exists, diversified both in diction and division, possessing subtile meanings logically combined, and gleaned from the Vedas, is a sacred work. Composed in elegant language, it includeth the subjects of other books. It is elucidated by other Shastras, and comprehendeth the sense of the four Vedas. We are desirous of hearing that history also called Bharata, the holy composition of the wonderful Vyasa, which dispelleth the fear of evil, just as it was cheerfully recited by the Rishi Vaisampayana, under the direction of Dwaipayana himself, at the snake-sacrifice of Raja Janamejaya?'

 "Sauti then said, 'Having bowed down to the primordial being Isana, to whom multitudes make offerings, and who is adored by the multitude; who is the true incorruptible one, Brahma, perceptible, imperceptible, eternal; who is both a non-existing and an existing-non-existing being; who is the universe and also distinct from the existing and non-existing universe; who is the creator of high and low; the ancient, exalted, inexhaustible one; who is Vishnu, beneficent and the beneficence itself, worthy of all preference, pure and immaculate; who is Hari, the ruler of the faculties, the guide of all things moveable and immoveable; I will declare the sacred thoughts of the illustrious sage Vyasa, of marvelous deeds and worshipped here by all. Some bards have already published this history, some are now teaching it, and others, in like manner, will hereafter promulgate it upon the earth. It is a great source of knowledge, established throughout the three regions of the world. It is possessed by the twice-born both in detailed and compendious forms. It is the delight of the learned for being embellished with elegant expressions, conversations human and divine, and a variety of poetical measures.

 In this world, when it was destitute of brightness and light, and enveloped all around in total darkness, there came into being, as the primal cause of creation, a mighty egg, the one inexhaustible seed of all created beings. It is called Mahadivya, and was formed at the beginning of the Yuga, in which we are told, was the true light Brahma, the eternal one, the wonderful and inconceivable being present alike in all places; the invisible and subtile cause, whose nature partaketh of entity and non-entity. From this egg came out the lord Pitamaha Brahma, the one only Prajapati; with Suraguru and Sthanu. Then appeared the twenty-one Prajapatis, viz., Manu, Vasishtha and Parameshthi; ten Prachetas, Daksha, and the seven sons of Daksha. Then appeared the man of inconceivable nature whom all the Rishis know and so the Viswe-devas, the Adityas, the Vasus, and the twin Aswins; the Yakshas, the Sadhyas, the Pisachas, the Guhyakas, and the Pitris. After these were produced the wise and most holy Brahmarshis, and the numerous Rajarshis distinguished by every noble quality. So the water, the heavens, the earth, the air, the sky, the points of the heavens, the years, the seasons, the months, the fortnights, called Pakshas, with day and night in due succession. And thus were produced all things which are known to mankind.
 """

In [43]:
with get_openai_callback() as cb:
    result = llm("Tell me a joke")
    result2 = llm("Tell me a joke")
    print(cb.total_tokens)

50


In [44]:
result

'\n\nQ: What did the fish say when it hit a concrete wall? \nA: Dam!'

In [45]:
result2

'\n\nQ: What did the fish say when it hit the wall?\nA: Dam!'

In [46]:
cb.total_cost

0.001

In [47]:
cb

Tokens Used: 50
	Prompt Tokens: 8
	Completion Tokens: 42
Successful Requests: 2
Total Cost (USD): $0.001